# Deep Learning for High Energy Particle Physics

## Importing Libraries

In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense

//anaconda/lib/python3.5/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
//anaconda/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)


## Reading in the Data

In [2]:
train = pd.read_csv('all_train.csv.gz')

In [3]:
train.head()

,# label,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f18,f19,f20,f21,f22,f23,f24,f25,f26,mass
0,1.0,-0.346368,0.416306,0.999236,0.475342,0.427493,-0.005984,1.989833,0.344530,1.566297,...,4.105282,0.267826,0.378718,1.743123,3.406367,4.350537,-0.352571,1.130032,2.227706,1000.0
1,1.0,1.708236,-0.319394,-1.241873,-0.887231,-0.871906,-0.005984,-0.001047,-1.038225,0.655748,...,-1.178141,-0.877361,-1.483769,-0.573682,-1.693781,-0.545062,-0.299118,-0.662942,-0.193019,750.0
2,0.0,-0.360693,1.794174,0.264738,-0.472273,-0.292344,-1.054221,-1.150495,1.423404,1.270098,...,-1.199511,0.539020,-1.590629,-0.573682,-0.543636,-0.937456,-0.300344,-0.523262,-1.506304,750.0
3,1.0,-0.377914,-0.103932,-0.649434,-2.125015,-1.643797,-0.005984,1.011112,-1.040340,-0.541991,...,0.463763,-0.006583,1.089122,-0.573682,-0.276348,-0.409272,-0.349926,-0.307123,0.529698,1250.0
4,0.0,-0.067436,-0.636762,-0.620166,-0.062551,1.588715,-0.005984,-0.595304,-1.238987,0.336844,...,-0.552837,-1.418494,-0.562982,1.743123,0.881802,0.002516,1.560950,-0.150760,-1.023889,750.0


## Scaling Mass

In [4]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
train['mass'] = scaler.fit_transform(train['mass'].reshape(-1,1))

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:3: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  app.launch_new_instance()


In [5]:
from sklearn.model_selection import train_test_split
X = train.drop(['# label'], axis=1)
y = train['# label']
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=42)

In [6]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4900000 entries, 4642813 to 6413414
Data columns (total 28 columns):
f0      float64
f1      float64
f2      float64
f3      float64
f4      float64
f5      float64
f6      float64
f7      float64
f8      float64
f9      float64
f10     float64
f11     float64
f12     float64
f13     float64
f14     float64
f15     float64
f16     float64
f17     float64
f18     float64
f19     float64
f20     float64
f21     float64
f22     float64
f23     float64
f24     float64
f25     float64
f26     float64
mass    float64
dtypes: float64(28)
memory usage: 1.1 GB


## Training a Deep Neural Network in Keras

In [11]:
from keras.regularizers import l2 # L2 regularization
from keras.callbacks import EarlyStopping

L2_CONSTANT = 1e-7 # L2 regularization constant
hidden_layer_sizes = [28, 28, 56, 56, 112, 56, 56, 28, 28]

# Model definition
model = Sequential()
first = True
for hidden_layer_size in hidden_layer_sizes:
    if first:
        model.add(Dense(hidden_layer_size, 
                        activation='sigmoid', 
                        input_dim=28, kernel_regularizer=l2(L2_CONSTANT)))
    else:
        model.add(Dense(hidden_layer_size, 
                        activation='sigmoid', 
                        kernel_regularizer=l2(L2_CONSTANT)))
        
model.add(Dense(1, activation='sigmoid'))

# callbacks
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=1, mode='min')

# compiling model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
print(model.summary())

# fitting model
model.fit(X_train.values, y_train.values, validation_data=(X_valid.values, y_valid.values), epochs=5, batch_size=32)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_25 (Dense)             (None, 28)                812       
_________________________________________________________________
dense_26 (Dense)             (None, 28)                812       
_________________________________________________________________
dense_27 (Dense)             (None, 56)                1624      
_________________________________________________________________
dense_28 (Dense)             (None, 56)                3192      
_________________________________________________________________
dense_29 (Dense)             (None, 112)               6384      
_________________________________________________________________
dense_30 (Dense)             (None, 56)                6328      
_________________________________________________________________
dense_31 (Dense)             (None, 56)                3192      
__________